_Author: Dmitry Bagaev_

In [ ]:
using DrWatson

In [ ]:
@quickactivate "RxInferThesisExperiments"

In [ ]:
using RxInferThesisExperiments, RxInfer, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

In [ ]:
const bfolder = datadir("lds", "rxinfer")
const outdir = plotsdir("lds")

In [ ]:
pgfplotsx()

In [ ]:
# Pregenerate paths for benchmark data
mkpath(bfolder);
mkpath(outdir);

In [ ]:
# Include the model specification
include(srcdir("models", "rxinfer", "rotating.jl"));

In [ ]:
function run_benchmark(params)
    @unpack T, d, seed = params
    
    environment = RotatingTracking(d; rng = StableRNG(seed))
    
    states, observations = rand(StableRNG(seed), environment, T);
    options  = (limit_stack_depth = 500, )
    model    = rotating(T, environment) 
    result   = inference(model = model, data = (y = observations, ), options = options);
    e_states = extract_posteriors(T, result)
    amse     = compute_amse(states, e_states)
    
    benchmark_modelcreation = @benchmark RxInfer.create_model(rotating($T, $(environment)))
    
    benchmark_inference = @benchmark inference(model = model, data = (y = observations, ), options = options) setup=begin
        model = rotating($T, $(environment))
        options  = (limit_stack_depth = 500, )
        states, observations = rand(StableRNG($seed), $environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local environment = RotatingTracking(d; rng = StableRNG(_seed))
        local states, observations = rand(StableRNG(_seed), environment, T);
        local options  = (limit_stack_depth = 500, )
        local model    = rotating(T, environment) 
        local result   = inference(model = model, data = (y = observations, ), options = options);
        local e_states = extract_posteriors(T, result)
        return compute_amse(states, e_states)
    end
    
    output = @strdict T d seed states e_states observations amse emse benchmark_modelcreation benchmark_inference
    
    return output
end

In [ ]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"    => [ 10, 20, 30, 50, 100, 200, 500, 1000, 2000, 5000, 10_000, 30_000, 50_000, 100_000 ],
    "d"    => [ 2, 3, 4, 8, 16 ],
    "seed" => [ 42 ]
));

In [ ]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

In [ ]:
benchmarks_table = sort(prepare_benchmarks_table(bfolder), [ :T ])

# Extra plots

In [ ]:
# `plotting`range
colors = ColorSchemes.tableau_10

# Some default settings for plotting
pfontsettings = (
    titlefontsize=18,
    guidefontsize=16,
    tickfontsize=14,
    legendfontsize=14,
    legend = :outertop,
    legend_font_halign = :left,
    legend_orientation=:h,
    legend_column = 2,
    size = (800, 300)
)

p1xticks = (
    [ 10, 100, 1000, 10_000, 100_000 ],
    [ L"10^1", L"10^2", L"10^3", L"10^4", L"10^5" ]
)

p1yticks = (
    [ 0.1, 1.0, 10.0, 100, 1000, 10000 ], 
    [ L"10^{-1}", L"10^{0}", L"10^{1}", L"10^{2}", L"10^{3}", L"10^{4}" ]
)

p1 = plot(
    xlabel = "Number of observations in dataset (log10-scale)", 
    ylabel = "Time (in ms, log10-scale)"; 
    xscale = :log10,
    yscale = :log10,
    xticks = p1xticks,
    yticks = p1yticks,
    ylims = (minimum(p1yticks[1]), maximum(p1yticks[1])),
    pfontsettings...
)

nd = [ 2, 4, 8, 16 ];
mshapes = [  :diamond, :circle, :rect, :utriangle ]
styles = [ :solid, :dash, :dot, :dashdot ]

for (index, (mshape, d)) in enumerate(zip(mshapes, nd))
    filtered    = filter((r) -> r["d"] == d, benchmarks_table)
    sorted      = sort(filtered, [ :T ])
    range       = map(f -> f["T"], eachrow(sorted))
    t_inference = map(f -> to_ms(f["inference"][1] - f["inference"][3]), eachrow(sorted))
    
    plot!(p1, range, t_inference, label = "$d dimensional", marker = mshape, color = colors[index], style = styles[index])
end

savefig(p1, joinpath(outdir, "02-rxinfer_rotating_scalability_size.tex"))
savefig(p1, joinpath(outdir, "02-rxinfer_rotating_scalability_size.pdf"))

##

p2xticks = (
    nd,
    string.(nd)
)

p2yticks = (
    [ 0.1, 1.0, 10.0, 100, 1000, 10_000 ], 
    [ L"10^{-1}", L"10^{0}", L"10^{1}", L"10^{2}", L"10^{3}", L"10^{4}" ]
)

p2 = plot(
    xlabel = "Number of dimensions", 
    ylabel = "Time (in ms, log10-scale)"; 
    yscale = :log10,
    xticks = p2xticks,
    yticks = p2yticks,
    ylims = (minimum(p2yticks[1]), maximum(p2yticks[1])),
    pfontsettings...
)

# :diamond, :hexagon, :cross, :xcross, :utriangle, :dtriangle, :rtriangle, :ltriangle, :pentagon, :heptagon,

Ts = [ 10, 1000, 10_000, 100_000 ];
mshapes = [:utriangle, :dtriangle, :rtriangle, :ltriangle ]
styles = [ :solid, :dash, :dot, :dashdot ]

for (index, (mshape, T)) in enumerate(zip(mshapes, Ts))
    filtered = filter((r) -> r["T"] == T, benchmarks_table)
    sorted      = filter((f) -> f["d"] ∈ nd, (sort(filtered, [ :d ])))
    range       = map(f -> f["d"], eachrow(sorted))
    t_inference = map(f -> to_ms(f["inference"][1] - f["inference"][3]), eachrow(sorted))
    
    plot!(p2, range, t_inference, label = "$T observations", marker = mshape, color = colors[index], style = styles[index])
end

savefig(p2, joinpath(outdir, "02-rxinfer_rotating_scalability_dims.tex"))
savefig(p2, joinpath(outdir, "02-rxinfer_rotating_scalability_dims.pdf"))

p = plot(p1, p2, size = (800, 600), layout = @layout([ a; b ]))

display("image/png", p)

# Versions

In [ ]:
versioninfo()

In [ ]:
] status